In [ ]:
import pygrib
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation
from mpl_toolkits.basemap import Basemap
import numpy as np
%matplotlib inline

In [ ]:
grbs = pygrib.open("../20130202.grb")
temp_layer = grbs[211]
print temp_layer
print temp_layer.values.shape
print map(lambda x: x.shape, temp_layer.latlons())
print temp_layer.latlons()[0][0,0]
print temp_layer.latlons()[1][0,0]
print temp_layer.latlons()[0][0,1]
print temp_layer.latlons()[1][0,1]
print temp_layer.latlons()[0][1,0]
print temp_layer.latlons()[1][1,0]

In [ ]:
# Ok, wtf is going on with the gap between july 25 and july 26?
grbs0725 = pygrib.open("../20130725.grb")
for layer in grbs0725:
    print layer

In [ ]:
grbs0726 = pygrib.open("../20130726.grb")
for layer in grbs0726:
    print layer
# Looks like  there's a "wind speed" layer here at 4 that trips stuff up. however, it gets 
# corrected by the end, so maybe it's just this layer that's put elsewhere. Very odd.

In [ ]:
# OK, so how do we uniquely identify the layer of interest? We want layer name = Temperature,
# regular_ll (eg typeOfLevel) = surface, and level = 0. See if this is a unique layer everywhere
grbs0725 = pygrib.open("../20130725.grb")
            
for i,layer in enumerate(grbs0725):
    if layer.name == "Temperature" and layer.typeOfLevel == "surface":
        if layer.level == 0:
            print "layer %d has all 3" % (i+1)
        else:
            print "layer %d has 2" % (i+1)
print 'done'

grbs0726 = pygrib.open("../20130726.grb")
            
for i,layer in enumerate(grbs0726):
    if layer.name == "Temperature" and layer.typeOfLevel == "surface":
        if layer.level == 0:
            print "layer %d has all 3" % (i+1)
        else:
            print "layer %d has 2" % (i+1)
print 'done'

In [ ]:
grbs0725 = pygrib.open("20130725.grb")
for i,layer in enumerate(grbs0725):
    #if layer.name == "Relative humidity" and layer.level == 0:# and layer.typeOfLevel == "unknown":
    #if layer.name == 'Surface air relative humidity':
    if "loud" in layer.name:
        print layer
# ok wtf is the difference
#layer1 = grbs0725[234]
#layer2 = grbs0725[273]
#for key in layer1.keys():
#    try:
#        if layer1[key] != layer2[key]:
#            print "%s\t%s\t%s" %(key, str(layer1[key]), str(layer2[key]))
#    except Exception as e:
#        print "failed because " + str(e)

In [ ]:
grbs0725 = pygrib.open("../20130725.grb")
            
for i,layer in enumerate(grbs0725):
    if layer.name == "Relative humidity" and layer.level == 0 \
        and layer.typeOfLevel == "unknown" and layer.typeOfFirstFixedSurface == '200':
        print "layer %d" % (i+1)
        print "toffs: " + str(layer.typeOfFirstFixedSurface == '200')

grbs0726 = pygrib.open("../20130726.grb")
            
for i,layer in enumerate(grbs0726):
    if layer.name == "Relative humidity" and layer.level == 0 \
        and layer.typeOfLevel == "unknown" and layer.typeOfFirstFixedSurface == '200':
        print "layer %d" % (i+1)
        print "toffs: " + str(layer.typeOfFirstFixedSurface)

In [ ]:
vals = layer1.values
lats,lons = layer1.latlons()
print "val shape: %s\tlat shape: %s\tlon shape: %s" %(str(vals.shape), str(lats.shape), str(lons.shape))
print lons[0,:]

In [ ]:
temp_vals = grbs0725.select(name='Temperature', typeOfLevel='surface')[0].values
ak_bb = [55,71,-165,-138]
plot_bb_0 = np.where(lats[:,0] <= ak_bb[0])[0][0]
plot_bb_1 = np.where(lats[:,0] <= ak_bb[1])[0][0]
plot_bb_2 = np.where(lons[0,:] >= (ak_bb[2] % 360))[0][0]
plot_bb_3 = np.where(lons[0,:] >= (ak_bb[3] % 360))[0][0]
plot_bb = [plot_bb_0, plot_bb_1, plot_bb_2, plot_bb_3]
print "plot_bb: " + str(plot_bb)
mp = Basemap(projection="merc",
              lat_0=55, lon_0=-165,
              llcrnrlat=55,
              llcrnrlon=-165,
              urcrnrlat=71,
              urcrnrlon=-138,
              resolution='i')
mp.drawcoastlines()
mp.imshow(temp_vals[plot_bb_0-1:plot_bb_1-1:-1, plot_bb_2:plot_bb_3])
mp.colorbar()

In [ ]:
plt.imshow(temp_vals[38:70,390:444])

In [ ]:
# Let's try our new function
import os
os.chdir('/Users/zbutler/research/fire_prediction')
import plotting.plot_map as pm
import cPickle
with open('data/temp_2013.pkl') as fpkl:
    temp_dict = cPickle.load(fpkl)
pm.draw_map_nogrid_static(ak_bb, month=1, day=9, gfs_dict=temp_dict)

In [ ]:
# try to make an animated thingy
def day2monthday(my_day, leapyear=False):
    if leapyear:
        month_arr = [31, 29, 31, 30, 31, 30, 31, 31, 30, 31, 30, 31]
    else:
        month_arr = [31, 28, 31, 30, 31, 30, 31, 31, 30, 31, 30, 31]
    month = 1
    days_left = my_day
    while days_left >= month_arr[month-1]:
        days_left -= month_arr[month-1]
        month += 1
    day = days_left + 1
    return month,day
    
def monthday2day(month, day, leapyear=False):
    """Convert month/day into days since Jan 1"""
    if leapyear:
        month_arr = [31, 29, 31, 30, 31, 30, 31, 31, 30, 31, 30, 31]
    else:
        month_arr = [31, 28, 31, 30, 31, 30, 31, 31, 30, 31, 30, 31]
    days = 0
    for mon in xrange(1, month):
        days += month_arr[mon - 1]
    days += day - 1
    return days

def update(i):
    monthday = day2monthday(i)
    temp_vals = temp_dict[monthday]
    im.set_data(temp_vals[plot_bb_0-1:plot_bb_1-1:-1, plot_bb_2:plot_bb_3])
    plt.title("Temps for %d/%d" %monthday)
    print "butts " + str(i)
    return im,
             
fig,ax = plt.subplots()
mp = Basemap(projection="merc",
              lat_0=55, lon_0=-165,
              llcrnrlat=55,
              llcrnrlon=-165,
              urcrnrlat=71,
              urcrnrlon=-138,
              resolution='i')
im = mp.imshow(temp_vals[plot_bb_0-1:plot_bb_1-1:-1, plot_bb_2:plot_bb_3], animated=True)
mp.drawcoastlines()
mp.colorbar()

anim = FuncAnimation(fig, update, frames=range(10,20), interval=200, blit=True)

In [ ]:
# animation round 2: save a bunch of pics and use mac's "convert" to turn them into a gif
mp = Basemap(projection="merc",
              lat_0=55, lon_0=-165,
              llcrnrlat=55,
              llcrnrlon=-165,
              urcrnrlat=71,
              urcrnrlon=-138,
              resolution='i')
start_day = monthday2day(6,1)
end_day = monthday2day(9,1)
min_temp = temp_dict['min']
max_temp = temp_dict['max']
for dayy in xrange(start_day, end_day):
    monthday = day2monthday(dayy)
    temp_vals = temp_dict[monthday]
    mp.imshow(temp_vals[plot_bb_0-1:plot_bb_1-1:-1, plot_bb_2:plot_bb_3],
              vmin=min_temp, vmax=max_temp)
    plt.title("Temps for %d/%d" %monthday)
    mp.drawcoastlines()
    mp.colorbar()
    plt.savefig('gifmaking/day%d.png' % dayy)
    plt.close()

os.system('convert -delay 100 -loop 0 gifmaking/day*.png gifmaking/temp_loop_2013.gif')
print "done"

In [ ]:
# Now let's do humidity
with open('data/hum_2013.pkl') as fpkl:
    hum_dict = cPickle.load(fpkl)
mp = Basemap(projection="merc",
              lat_0=55, lon_0=-165,
              llcrnrlat=55,
              llcrnrlon=-165,
              urcrnrlat=71,
              urcrnrlon=-138,
              resolution='i')
start_day = monthday2day(6,1)
end_day = monthday2day(9,1)
min_hum = hum_dict['min']
max_hum = hum_dict['max']
for dayy in xrange(start_day, end_day):
    monthday = day2monthday(dayy)
    hum_vals = hum_dict[monthday]
    mp.imshow(hum_vals[plot_bb_0-1:plot_bb_1-1:-1, plot_bb_2:plot_bb_3],
              vmin=min_hum, vmax=max_hum)
    plt.title("Humidity for %d/%d" %monthday)
    mp.drawcoastlines()
    mp.colorbar()
    plt.savefig('gifmaking/day%d.png' % dayy)
    plt.close()

os.system('convert -delay 100 -loop 0 gifmaking/day*.png gifmaking/humidity_loop_2013.gif')
print "done"

In [ ]:
# let's try drawing gridlines
meridians = np.arange(ak_bb[2], ak_bb[3], 1)  # 111km in a degree
parallels = np.arange(ak_bb[0], ak_bb[1], 1)  # 111km in a degree
mp = Basemap(projection="merc",
              lat_0=55, lon_0=-165,
              llcrnrlat=55,
              llcrnrlon=-165,
              urcrnrlat=71,
              urcrnrlon=-138,
              resolution='i')
temp_vals = temp_dict[(6,9)]
mp.imshow(temp_vals[plot_bb_0-1:plot_bb_1-1:-1, plot_bb_2:plot_bb_3],
              vmin=min_temp, vmax=max_temp)
mp.drawparallels(parallels)
mp.drawmeridians(meridians)
mp.drawcoastlines()
mp.colorbar()
plt.title("Temperature for 6/09")
plt.show()

In [ ]:
with open("data/modis.pkl") as fpd:
    modis = cPickle.load(fpd)

In [ ]:
# individual fire plot with grid
from IPython import display
fire_bb = [64.6, 64.9, -147, -146.4]
meridians = np.arange(fire_bb[2], fire_bb[3], 1.1/111)  # 111km in a degree
parallels = np.arange(fire_bb[0], fire_bb[1], 1.1/111)  # 111km in a degree
mp = Basemap(projection="merc",
              lat_0=float(fire_bb[0]+fire_bb[1])/2, lon_0=float(fire_bb[2]+fire_bb[3])/2,
              llcrnrlat=fire_bb[0],
              llcrnrlon=fire_bb[2],
              urcrnrlat=fire_bb[1],
              urcrnrlon=fire_bb[3],
              resolution='i')
my_fires = modis[(modis.lat < fire_bb[1]) & (modis.lat > fire_bb[0]) &
                 (modis.long < fire_bb[3]) & (modis.long > fire_bb[2])]
month = 6
day = 25
prev_lats = []
prev_longs = []
while month < 8 or day < 12:
    todays_fires = my_fires[(my_fires.year == 2013) & (my_fires.month == month) &
                            (my_fires.day == day)]
    display.clear_output(wait=True)
    plt.figure(figsize=(5,10))
    mp.drawcoastlines()
    if len(prev_lats):
        mp.plot(prev_longs, prev_lats, 'ko')
    if len(todays_fires):
        today_longs, today_lats = mp(np.array(todays_fires.long), np.array(todays_fires.lat))
        mp.plot(today_longs, today_lats, 'ro')
        prev_lats += list(today_lats)
        prev_longs += list(today_longs)
    mp.drawparallels(parallels)
    mp.drawmeridians(meridians)
    plt.title("Fires in 5km BB on %d %d" %(month, day))
    plt.show()
    raw_input("hit enter")
    day += 1
    if day >= 31 or (day >= 30 and month == 6):
        day = 1
        month += 1

In [ ]:
import matplotlib.patches as patches
# individual fire plot with grid and cells
fire_bb = [64.6, 64.9, -147, -146.4]
meridians = np.arange(fire_bb[2], fire_bb[3], 1.1/111)  # 111km in a degree
parallels = np.arange(fire_bb[0], fire_bb[1], 1.1/111)  # 111km in a degree
my_fires = modis[(modis.lat < fire_bb[1]) & (modis.lat > fire_bb[0]) &
                 (modis.long < fire_bb[3]) & (modis.long > fire_bb[2])]

In [ ]:
mp = Basemap(projection="merc",
              lat_0=float(fire_bb[0]+fire_bb[1])/2, lon_0=float(fire_bb[2]+fire_bb[3])/2,
              llcrnrlat=fire_bb[0],
              llcrnrlon=fire_bb[2],
              urcrnrlat=fire_bb[1],
              urcrnrlon=fire_bb[3])
month = 6
day = 25
prev_lat_cells = []
prev_long_cells = []
while month < 8 or day < 12:
    todays_fires = my_fires[(my_fires.year == 2013) & (my_fires.month == month) &
                            (my_fires.day == day)]
    display.clear_output(wait=True)
    plt.figure(figsize=(5,10))
    mp.drawcoastlines()
#    if len(prev_lat_cells):
#        mp.plot(prev_long_cells, prev_lat_cells, 'ko')
    if len(todays_fires):
        today_longs, today_lats = np.array(todays_fires.long), np.array(todays_fires.lat)
        print 'yolo'
        for lon,lat in zip(today_longs, today_lats):
            lon_cell = np.where(meridians > lon)[0][0]
            lat_cell = np.where(parallels > lat)[0][0]
            plotlat, plotlon = mp(lon, lat)
            mp.plot(plotlon, plotlat, 'r+')
            print "plotlat, plotlon " + str((plotlat, plotlon))
            rect = patches.Rectangle(mp(meridians[lon_cell-1], parallels[lat_cell-1]), 1.1/111, 1.1/111, facecolor='red')
            print rect
            print mp(meridians[-10:], parallels[-10:])
            plt.gca().add_patch(rect)
            prev_lat_cells.append(lat_cell)
            prev_long_cells.append(lon_cell)
            break
        print 'yolo 2'
    mp.drawparallels(parallels)
    mp.drawmeridians(meridians)
    plt.title("Fires in 5km BB on %d %d" %(month, day))
    plt.show()
    raw_input("hit enter")
    day += 1
    if day >= 31 or (day >= 30 and month == 6):
        day = 1
        month += 1

In [ ]:
# time dependent plot
with open("modis.pkl") as fpkl:
    modis = cPickle.load(fpkl)
    
fire_bb = [64.6, 64.9, -147, -146.4]
meridians = np.arange(fire_bb[2], fire_bb[3], 1.1/111)  # 111km in a degree
parallels = np.arange(fire_bb[0], fire_bb[1], 1.1/111)  # 111km in a degree
mp = Basemap(projection="merc",
              lat_0=float(fire_bb[0]+fire_bb[1])/2, lon_0=float(fire_bb[2]+fire_bb[3])/2,
              llcrnrlat=fire_bb[0],
              llcrnrlon=fire_bb[2],
              urcrnrlat=fire_bb[1],
              urcrnrlon=fire_bb[3],
              resolution='i')
my_fires = modis[(modis.lat < fire_bb[1]) & (modis.lat > fire_bb[0]) &
                 (modis.long < fire_bb[3]) & (modis.long > fire_bb[2])]
month = 6
day = 25
prev_lats = []
prev_longs = []
while month < 8 or day < 12:
    todays_fires = my_fires[(my_fires.year == 2013) & (my_fires.month == month) &
                            (my_fires.day == day)]
    display.clear_output(wait=True)
    plt.figure(figsize=(5,10))
    mp.drawcoastlines()
    if len(prev_lats):
        mp.plot(prev_longs, prev_lats, 'ko')
    if len(todays_fires):
        today_longs, today_lats = mp(np.array(todays_fires.long), np.array(todays_fires.lat))
        today_times = np.array(today_fires.hour)
        mp.plot(today_longs, today_lats, c=today_times)
        prev_lats += list(today_lats)
        prev_longs += list(today_longs)
    mp.drawparallels(parallels)
    mp.drawmeridians(meridians)
    plt.title("Fires in 5km BB on %d %d" %(month, day))
    plt.show()
    raw_input("hit enter")
    day += 1
    if day >= 31 or (day >= 30 and month == 6):
        day = 1
        month += 1
